In [19]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import norm
import tqdm 

def fi(mu_i, i, mu_minus_i, sigma, gamma, X, Y, lamb):
    f = mu_i * (X.T @ X)[i, np.arange(X.shape[1])!=i] @ (gamma[np.arange(len(gamma))!=i] * mu_minus_i)
    f += 0.5*(X.T @ X)[i, i] * mu_i**2
    f -= (Y.T @ X)[i] * mu_i
    f += lamb * sigma[i] * np.sqrt(2/np.pi) * np.exp(-mu_i**2/(2*sigma[i]**2))
    f += lamb * mu_i * (1 - 2*norm.cdf(-mu_i/sigma[i]))

    return f


def gi(sigma_i, i, mu, X, lamb):

    g = 0.5 * (X.T @ X)[i, i] * sigma_i**2
    g += lamb * mu[i] * sigma_i * np.sqrt(2/np.pi) * np.exp(-mu[i]**2/(2*sigma_i**2))
    g += lamb * mu[i] * (1 - norm.cdf(mu[i]/sigma_i))
    g += - np.log(sigma_i)
    return g

def Gamma_function(i, mu, sigma, gamma, X, Y, a0, b0, lamb):
    Gamma = np.log(a0/b0)
    Gamma += np.log(np.sqrt(np.pi/2)*sigma[i]*lamb)
    Gamma -= mu[i] * (X.T @ X)[i, np.arange(X.shape[1])!=i] @ (gamma[np.arange(len(gamma))!=i] * mu[np.arange(len(mu))!=i])
    Gamma -= 0.5*(X.T @ X)[i, i] * (mu[i]**2 + sigma[i]**2)
    Gamma += (Y.T @ X)[i] * mu[i]
    Gamma -= lamb * sigma[i] * np.sqrt(2/np.pi) * np.exp(-mu[i]**2/(2*sigma[i]**2))
    Gamma -= lamb * mu[i] * (1 - 2*norm.cdf(-mu[i]/sigma[i]))

    return Gamma + 0.5


def H(p, ent):
    for j in range(len(p)):
        if p[j] < 1 - 1e-10 and p[j] > 1e-10:
            ent[j] = -p[j]*np.log(p[j]) - (1-p[j])*np.log(1-p[j])
    return ent


def mu_0(X, Y):
    _, p = X.shape
    return np.linalg.inv(X.T @ X + np.eye(p)) @ X.T @ Y


def inv_logit(p):
    if p > 0:
        return 1. / (1. + np.exp(-p))
    elif p <= 0:
        return np.exp(p) / (1 + np.exp(p))
    else:
        print("AWPER")
        raise ValueError


def variational_bayes(X, Y, sigma, gamma, mu, a0, b0, lamb, eps=1e-5, max_it=1):
    deltaH = 10
    p = len(mu)
    it = 0
    a = np.argsort(np.abs(mu))
    pbar = tqdm.tqdm(total=max_it)
    ent = H(gamma, np.random.random(p))
    while it < max_it and deltaH >= eps:
        pbar.update(1)
        for j in range(p):
            i = a[j]
            ## update mu_i
            mu_minus_i = mu[np.arange(len(mu))!=i]
            res = minimize(fi, mu[i], args=(i, mu_minus_i, sigma, gamma, X, Y, lamb))
            mu[i] = res.x

            ## update gamma_i
            cons = ({'type': 'ineq', 'fun': lambda x:  x})
            res = minimize(gi, sigma[i], args=(i, mu, X, lamb), constraints=cons)
            sigma[i] = res.x

            ## update gamma
            Gamma = Gamma_function(i, mu, sigma, gamma, X, Y, a0, b0, lamb)
            gamma[i] = inv_logit(Gamma)

        it += 1
        ent_old = np.copy(ent)
        ent = H(gamma, ent)
        deltaH = np.max(np.abs(ent) - np.abs(ent_old))
        print(deltaH)
    pbar.close( )
    return mu, sigma, gamma








In [20]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()


In [21]:
n, p, s = 100, 200, 20

theta = 10*np.ones(p)
theta[:s] = 0
X = np.random.normal(0, 1, size=(n, p))
Y = X @ theta
mu = mu_0(X, Y)
sigma = 10*np.random.random(p)
gamma = np.random.random(p)
mu, sigma, gamma = variational_bayes(X, Y, sigma, gamma, mu, a0=1, b0=p, lamb=1, eps=1e-5, max_it=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]<ipython-input-19-24dd7331de20>:21: RuntimeWarning: invalid value encountered in log
  g += - np.log(sigma_i)
<ipython-input-19-24dd7331de20>:21: RuntimeWarning: invalid value encountered in log
  g += - np.log(sigma_i)
<ipython-input-19-24dd7331de20>:21: RuntimeWarning: invalid value encountered in log
  g += - np.log(sigma_i)
<ipython-input-19-24dd7331de20>:19: RuntimeWarning: divide by zero encountered in true_divide
  g += lamb * mu[i] * sigma_i * np.sqrt(2/np.pi) * np.exp(-mu[i]**2/(2*sigma_i**2))
<ipython-input-19-24dd7331de20>:20: RuntimeWarning: divide by zero encountered in true_divide
  g += lamb * mu[i] * (1 - norm.cdf(mu[i]/sigma_i))
<ipython-input-19-24dd7331de20>:21: RuntimeWarning: divide by zero encountered in log
  g += - np.log(sigma_i)
<ipython-input-19-24dd7331de20>:21: RuntimeWarning: invalid value encountered in log
  g += - np.log(sigma_i)
<ipython-input-19-24dd7331de20>:19: RuntimeWarning: divide by zero encountered in tru

0.5108917659449239


  0%|          | 3/1000 [00:12<1:07:02,  4.03s/it]

0.05474043554878347


<ipython-input-19-24dd7331de20>:21: RuntimeWarning: invalid value encountered in log
  g += - np.log(sigma_i)
<ipython-input-19-24dd7331de20>:21: RuntimeWarning: invalid value encountered in log
  g += - np.log(sigma_i)
  0%|          | 4/1000 [00:17<1:14:00,  4.46s/it]

...

1.8836530044166917e-05


  5%|▍         | 48/1000 [04:21<1:22:10,  5.18s/it]

1.0393247140976988e-05


  5%|▍         | 48/1000 [04:27<1:28:19,  5.57s/it]

5.8650242976748465e-06


In [22]:
np.unique(gamma == 1, return_counts=True)

(array([False,  True]), array([ 20, 180]))

In [25]:
theta = gamma * np.random.normal(mu, sigma)

In [31]:
(X @ theta)[:10]

array([ 142.79439414,   74.7456955 ,   91.51712976,  244.39949731,
       -162.30024156,  -54.54658436,  -21.93495754,  -99.01473075,
        238.943847  ,  120.06397042])

In [32]:
Y[:10]

array([ 142.84459278,   76.49698088,   91.57275278,  243.28727912,
       -163.16451502,  -52.60007252,  -22.5311845 ,  -97.93486912,
        239.14541256,  119.37570615])